In [1]:
import pandas as pd

def filter_df(df, prefix=""):
    return df[(df['Origin'] != df[f'{prefix}choice']) | (df['Origin'] != df[f'{prefix}flipped_choice'])]

def fail_check(df, prefix=""):
    return df[(df['Origin'] == df[f'{prefix}choice']) & (df['Origin'] == df[f'{prefix}flipped_choice'])]

In [2]:
# dataset = "full"
dataset = "long"

In [3]:
df = pd.read_csv(f'data/failed_distinguishes_for_llama3.1-70B-{dataset}.csv')
assert len(fail_check(df)) == 0
df['rows_failed'] = 1
print(len(df.groupby(by=['w_str', 'm_str', 'prompt_type', 'Origin', 'attack1_id', 'attack2_id']).agg({'rows_failed': 'sum'})))
df = df.groupby(by=['m_str', 'w_str', 'prompt_type', 'Origin', 'attack1_id', 'attack2_id']).agg({'rows_failed': 'sum'}).reset_index()
display(df)
df['groups'] = 1
display(df.groupby(by=['m_str', 'w_str']).agg({'groups': 'sum'}))
display(df.groupby(by=['prompt_type']).agg({'groups': 'sum'}))

13


,m_str,w_str,prompt_type,Origin,attack1_id,attack2_id,rows_failed
0,SentenceMutator,Adaptive,space,A,3,4,10
1,SentenceMutator,Adaptive,space,A,3,5,10
2,SentenceMutator,KGW,paris,A,1,2,5
3,SentenceMutator,KGW,paris,B,0,1,8
4,SentenceMutator,KGW,space,A,3,4,9
5,SentenceMutator,SIR,paris,A,0,2,16
6,SentenceMutator,SIR,paris,A,1,2,5
7,SentenceMutator,SIR,paris,B,0,1,5
8,SentenceMutator,SIR,paris,B,0,2,5
9,SentenceMutator,SIR,space,A,3,5,17


groups
m_str           w_str           
SentenceMutator Adaptive       2
                KGW            3
                SIR            8

,groups
prompt_type,
paris,6
space,7


In [4]:
df = pd.read_csv(f'results/redemption_{dataset}_with_4o_logic.csv')
# filter df for failed rows
df = filter_df(df, prefix="2nd_")
print(len(df))
df.to_csv(f'data/failed_redemption_{dataset}_with_4o_logic.csv')
df['rows_failed'] = 1
df.groupby(by=['m_str', 'w_str', 'prompt_type', 'Origin', 'attack1_id', 'attack2_id']).agg({'rows_failed': 'sum'}).reset_index()

22


,m_str,w_str,prompt_type,Origin,attack1_id,attack2_id,rows_failed
0,SentenceMutator,Adaptive,space,A,3,4,4
1,SentenceMutator,SIR,space,A,3,5,12
2,SentenceMutator,SIR,space,B,3,5,6


In [5]:
df.drop_duplicates(subset=['m_str', 'w_str', 'prompt_type', 'Origin', 'attack1_id', 'attack2_id'], keep='last', inplace=True)
df = df[['prompt', 'P', 'origin_A', 'origin_B']]
df.to_csv(f'data/failed_redemption_{dataset}_with_4o_logic_no_choices.csv', index=False)

In [6]:
row = df.iloc[1]
P = row['P']
origin_A = row['origin_A']
origin_B = row['origin_B']

print(f'''Modified text: {P}
___________________________________________________________
Original response A: {origin_A}
___________________________________________________________
Original response B: {origin_B}
___________________________________________________________
Which response does the modified text come from?

Reply with only one character.
''')

Modified text: The cost of space exploration is often seen as an unsustainable drain on public funds, providing relatively meager returns on investment. Advances in our comprehension of the cosmos have yielded profound discoveries, profoundly impacting our daily routines and providing a beacon of optimism for tackling some of humanity's most critical challenges. Breakthroughs in technology have profoundly impacted humanity's comprehension, fueled by the pioneering achievements of space exploration. A widely used material in mattresses was initially created to improve the safety of airplane seats. Technologies used in the International Space Station's water recycling system are being repurposed globally to ensure that communities have a reliable access to clean drinking water. As space exploration advances, its impact is becoming more pronounced on Earth, turning a once-distant protective measure into an essential component of our daily routines. Advances in space exploration have creat

In [7]:
print(f'''Modified text: {P}
___________________________________________________________
Original response A: {origin_B}
___________________________________________________________
Original response B: {origin_A}
___________________________________________________________
Which response does the modified text come from?

Reply with only one character.
''')

Modified text: The cost of space exploration is often seen as an unsustainable drain on public funds, providing relatively meager returns on investment. Advances in our comprehension of the cosmos have yielded profound discoveries, profoundly impacting our daily routines and providing a beacon of optimism for tackling some of humanity's most critical challenges. Breakthroughs in technology have profoundly impacted humanity's comprehension, fueled by the pioneering achievements of space exploration. A widely used material in mattresses was initially created to improve the safety of airplane seats. Technologies used in the International Space Station's water recycling system are being repurposed globally to ensure that communities have a reliable access to clean drinking water. As space exploration advances, its impact is becoming more pronounced on Earth, turning a once-distant protective measure into an essential component of our daily routines. Advances in space exploration have creat

In [8]:
"""
Results for GPT-o3-mini-high on last row of each failed group in full
B, A
A, B
B, A
B, A
"""
pass

In [9]:
"""
Results for GPT-o3-mini-high on last row of each failed group in long
A, B
A, B
B, A
"""
pass